In [2]:
import os
import re
import numbers

import pandas as pd
import numpy as np

In [3]:
PROJECT_FOLDER = os.path.abspath('..')

# 1. Watching files:
### 1.1 data/result_task3.csv

In [3]:
result_example = pd.read_csv(f'{PROJECT_FOLDER}/data/result_task3.csv', sep=';')
result_example

,user_id,book_id_1,book_id_2,book_id_3,book_id_4,book_id_5
0,111000,12,16,10232,2342,234
1,777333,123,754,232,234243,2323
2,8888,1223,2334,2335,234545,111222


### 1.2 data/dataset_knigi_1.xlsx

In [19]:
books_data = pd.read_excel(f'{PROJECT_FOLDER}/data/dataset_knigi_1.xlsx', sheet_name=None)
books_data = books_data['Лист1']
books_data['id'] = books_data['source_url'].apply(lambda x: int(x.split('/')[-2]))
books_data.drop(columns=['source_url'], inplace=True, axis=1)
books_data.head()

,event,dt,user_id,id
0,add,2021-06-01,1,1163786
1,add,2021-06-01,1,66612
2,add,2021-06-01,1,1034596
3,add,2021-06-01,1,1152760
4,add,2021-06-01,1,16168


In [23]:
books_data.to_csv(f'{PROJECT_FOLDER}/data/user_book_interaction.csv', index=False)

In [24]:
unique_books_ids = np.unique(books_data['id'])
create_order__unique_books_id = list(books_data[books_data['event'] == 'create_order']['id'].value_counts().index)
add__unique_books_id = list(books_data[books_data['event'] == 'add']['id'].value_counts().index)

### 1.3 data/books.jsn

In [4]:
books_info = pd.read_json(f'{PROJECT_FOLDER}/data/raw_data/books.jsn')
books_info

,id,parentId,bbk,title,year,parentAuthor,contribution,parentNumberOfPart,isbn,annotation,...,rubric,author,publisher,place,serial,f_deleted,libraryId,catalogueId,start_timestamp,end_timestamp
0,1272541,0,84(2Рос=Рус)1,Бесы,2013,0,Федор Достоевский,,978-5-17-063255-8,"""Бесы""- одно из наиболее трагических, загадочн...",...,0,0,0,0,0,False,0,0,0,0
1,1960078,0,84(4Вел),На службе зла : роман,2017,0,Роберт Гэлбрейт ; перевод с английского Е. Пет...,,978-5-389-12682-4,Робин Эллакотт получает с курьером таинственны...,...,0,0,0,0,0,False,0,0,0,0
2,26991,0,84(7Сое),"Жареные зеленые помидоры в кафе ""Полустанок"" :...",2004,0,Фэнни Флэгг ; [пер. с англ. Д. Крупской],,5-7516-0310-9,Роман американской писательницы Ф.Флэгг расска...,...,0,0,0,0,0,False,0,0,0,0
3,1781433,0,84(7Сое),Эмпайр Фоллз : роман,2020,0,Ричард Руссо ; перевод с английского Елены Пол...,,978-5-86471-807-0,"Майлз Роби двадцать лет готовит бургеры в ""Имп...",...,0,0,0,0,0,False,0,0,0,0
4,1687060,0,84(7США),Эмпайр Фоллз : роман,2019,0,Р. Руссо ; пер. Е. Полецкая,,978-5-86471-807-0,,...,0,0,0,0,0,False,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4677,814384,0,26.89(2-2М),Административные округа Москвы:Культура.Истори...,1998,0,Глушкова В.Г.,,5-691-00168-X,В книге дана обобщенная краеведческая и социал...,...,0,0,0,0,0,False,0,0,0,0
4678,840090,0,26.89(2-2М)я2,Путеводитель Москва 850:События в юбилейные дни,1997,0,,,,"Путеводитель по Москве, рассказывающий о ее ис...",...,0,0,0,0,0,False,0,0,0,0
4679,727406,0,84,Трое в лодке (не считая собаки),1958,0,"Пер. М.Донского,Э.Линецкой",,,,...,0,0,0,0,0,False,0,0,0,0
4680,305891,0,85.118(2-2М),Проспекты советской Москвы. История реконструк...,2014,0,Алексей Рогачев,,978-5-227-05721-1,Лучами расходятся от центра города радиальные ...,...,0,0,0,0,0,False,0,0,0,0


#### 1.3.1 data leak???

In [7]:
print("DIFF BETWEEN 'data/dataset_knigi_1.xlsx' AND 'data/books.jsn' unique number of books.")
all_books_xlsx = np.unique(books_info['id'])
len(set(all_books_xlsx) - set(unique_books_ids))

DIFF BETWEEN 'data/dataset_knigi_1.xlsx' AND 'data/books.jsn' unique number of books.


743

#### 1.3.2 Preprocessing

In [8]:
columns_delete = ['parentNumberOfPart', 'parentId', 'parentAuthor', 'isbn', 'issn', 'deleted', 'titleOfPart', 'nextAuthor',
                  'edition', 'bibliographyLevel', 'numberOfPart', 'parentTitle', 'parentAdditionalInfo', 'parentPlace', 'parentPublisher',
                  'parentYear', 'changed', 'person', 'parentEdition', 'udk', 'cover', 'node', 'language_id', 'material_id', 'ageRestriction_id',
                  'rubric_id', 'rubric_parentId', 'author_info', 'author_dates', 'author_surname', 'author_names', 'author_initials',
                  'author_fullName', 'author_fullNameAlt', 'publisher_id', 'publisher_name', 'place_id', 'place_name', 'serial_id',
                  'year_value', 'common_search_field', 'totalOutCount', 'totalInplaceCount', 'freeCount', 'freeHands', 'freeOnline',
                  'orderedCount', 'outputCount', 'smart_collapse_field', 'author_surname_for_sort', 'author_surname_language_level',
                  'title_for_sort', 'title_language_level', 'publisher_name_for_sort', 'publisher_name_language_level', 'serial_name_for_sort',
                  'serial_name_language_level', 'dates', 'info', 'initials', 'eiskId', 'fixed', 'pointEiskId', 'language', 'material', 'ageRestriction',
                  'rubric', 'author', 'publisher', 'place', 'serial', 'f_deleted', 'libraryId', 'catalogueId', 'start_timestamp', 'end_timestamp', 'siglaEiskId',
                  'libraryAvailability']
books_info.drop(axis=1, columns=columns_delete, inplace=True)
books_info.head()


,id,bbk,title,year,contribution,annotation,keyword,titleAdditionalInfo,volume,parallelTitle,publicationType,language_name,material_name,ageRestriction_name,rubric_name,author_id,serial_name,collapse_field,title_orig,available
0,1272541,84(2Рос=Рус)1,Бесы,2013,Федор Достоевский,"""Бесы""- одно из наиболее трагических, загадочн...",роман,,"604, [2] с.",,mono,Русский,Текст,None,Художественная литература,45567.0,Русская классика,8b7355ec9ebf6ae5fdb84f917be6cc54,Бесы,2
1,1960078,84(4Вел),На службе зла : роман,2017,Роберт Гэлбрейт ; перевод с английского Е. Пет...,Робин Эллакотт получает с курьером таинственны...,,роман,"540, [1] с.",,mono,Русский,Текст,16+,Художественная литература,41515.0,Иностранная литература. Современная классика,0dc0c51c9949ae797a04f1622dffb0f9,На службе зла,1
2,26991,84(7Сое),"Жареные зеленые помидоры в кафе ""Полустанок"" :...",2004,Фэнни Флэгг ; [пер. с англ. Д. Крупской],Роман американской писательницы Ф.Флэгг расска...,,роман,"428, [1] с.",,mono,Русский,Текст,None,Художественная литература,116027.0,None,63e4b98dbc52b93a8d52540c1bed7c15,"Жареные зеленые помидоры в кафе ""Полустанок""",7
3,1781433,84(7Сое),Эмпайр Фоллз : роман,2020,Ричард Руссо ; перевод с английского Елены Пол...,"Майлз Роби двадцать лет готовит бургеры в ""Имп...",,роман,"606, [1] с.",,mono,Русский,Текст,16+,Художественная литература,141830.0,None,3333ebc7e54b7aab6b8e65f30cb2b4f4,Эмпайр Фоллз,11
4,1687060,84(7США),Эмпайр Фоллз : роман,2019,Р. Руссо ; пер. Е. Полецкая,,,роман,608с.,,mono,Русский,Текст,None,Художественная литература,141830.0,None,0f4b28cf531daa0af4662a85110fb249,Эмпайр Фоллз,1


- id: id of book, (there could be the same books, with different id)
- bbk: classic RUSSIAN classification of books
- year: year of book
- contribution: author
- annotation: short description
- keyword: genre of book
- titleAdditionalInfo: additional genre for book
- volume: number of pages, disks
- parallelTitle (95% missed data): title on English language
- publicationType: one of: ['mono', 'volume', 'edition'], commonly 'mono' 99%
- language_name: language name of book
- material_name: one of: ['text', 'braily text', 'audio', ...]
- ageRestriction_name: restriction age [4+, 16+, 18+]
- rubric_name: additional genre for book
- author_id: author id
- serial_name: additional genre for book
- collapse_field: hash-sum for book, unique identification. It is plenty without physical duplicates.
- title_orig: additional title
- available: available to borrow

In [9]:
def check_column(dataset: pd.DataFrame, column: str = None):
    if column is None:
        series = dataset
    else:
        series = dataset[column]

    number_nan = sum(series.isna())
    empty_str = len(series == '')
    total_len = len(series)
    duplicates = sum(series.duplicated())

    _print = [arr / total_len for arr in [number_nan, empty_str, duplicates]]
    _text = ['NaN', 'empty_str', 'duplicates']

    for n, t in zip(_print, _text):
        print(f'{n}% of {t}.', end='\n')

    print(f'\n{series.describe()}\n')

    return series.value_counts()

In [10]:
check_column(books_info, 'title_orig')

0.0% of NaN.
1.0% of empty_str.
0.13135412217001283% of duplicates.

count     4682
unique    4067
top           
freq        45
Name: title_orig, dtype: object



                           45
Энциклопедия для детей     19
Избранные произведения      9
Собрание сочинений          7
Думай                       7
                           ..
Дом с волшебными окнами     1
Валентина Терешкова         1
Старик-годовик              1
Белая юрта                  1
История Москвы              1
Name: title_orig, Length: 4067, dtype: int64

In [11]:
books_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4682 entries, 0 to 4681
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   4682 non-null   int64  
 1   bbk                  4682 non-null   object 
 2   title                4682 non-null   object 
 3   year                 4678 non-null   object 
 4   contribution         4682 non-null   object 
 5   annotation           4682 non-null   object 
 6   keyword              4682 non-null   object 
 7   titleAdditionalInfo  4682 non-null   object 
 8   volume               4682 non-null   object 
 9   parallelTitle        4682 non-null   object 
 10  publicationType      4682 non-null   object 
 11  language_name        4585 non-null   object 
 12  material_name        4682 non-null   object 
 13  ageRestriction_name  2537 non-null   object 
 14  rubric_name          4597 non-null   object 
 15  author_id            4298 non-null   f

In [12]:
def year_preprocessing(x):
    if x is None:
        return 999999
    number = re.search(r'\d+', x)
    if number is None:
        return 999999
    number = int(number.group())
    if len(str(number)) < 4:
        return 999999
    return number

def volume_preprocessing(x):
    number = re.search(r'\d+', x)
    if number is None:
        return 999999
    number = int(number.group())
    if number < 3:
        return 999999
    return number

def age_restriction_preprocessing(x):
    if x is None:
        return None
    number = re.search(r'\d+', x)
    if number is None:
        return None
    return int(number.group())

def available_preprocessing(x):
    if x is None:
        return 0
    if isinstance(x, numbers.Integral):
        return x
    number = re.search(r'\d+', x)
    if number is None:
        return 0
    return int(number.group())

In [13]:
books_info['year'] = pd.to_numeric(books_info['year'].apply(year_preprocessing))
books_info['volume'] = pd.to_numeric(books_info['volume'].apply(volume_preprocessing))
books_info['ageRestriction_name'] = pd.to_numeric(books_info['ageRestriction_name'].apply(age_restriction_preprocessing))
books_info['available'] = pd.to_numeric(books_info['available'].apply(available_preprocessing))

In [14]:
books_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4682 entries, 0 to 4681
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   4682 non-null   int64  
 1   bbk                  4682 non-null   object 
 2   title                4682 non-null   object 
 3   year                 4682 non-null   int64  
 4   contribution         4682 non-null   object 
 5   annotation           4682 non-null   object 
 6   keyword              4682 non-null   object 
 7   titleAdditionalInfo  4682 non-null   object 
 8   volume               4682 non-null   int64  
 9   parallelTitle        4682 non-null   object 
 10  publicationType      4682 non-null   object 
 11  language_name        4585 non-null   object 
 12  material_name        4682 non-null   object 
 13  ageRestriction_name  2537 non-null   float64
 14  rubric_name          4597 non-null   object 
 15  author_id            4298 non-null   f

In [15]:
def remove_duplicate_on_hash(dataset: pd.DataFrame) -> pd.DataFrame:
    unique_fields = np.unique(dataset['collapse_field'])

    for u_f in unique_fields:
        slice = dataset[dataset['collapse_field'] == u_f]
        total_available = sum(slice['available'])
        dataset.loc[dataset['collapse_field'] == u_f, 'available'] = total_available

    dataset.drop_duplicates(['collapse_field'], inplace=True)

    return dataset


In [16]:
books_info = remove_duplicate_on_hash(books_info)
books_info.head()

,id,bbk,title,year,contribution,annotation,keyword,titleAdditionalInfo,volume,parallelTitle,publicationType,language_name,material_name,ageRestriction_name,rubric_name,author_id,serial_name,collapse_field,title_orig,available
0,1272541,84(2Рос=Рус)1,Бесы,2013,Федор Достоевский,"""Бесы""- одно из наиболее трагических, загадочн...",роман,,604,,mono,Русский,Текст,NaN,Художественная литература,45567.0,Русская классика,8b7355ec9ebf6ae5fdb84f917be6cc54,Бесы,2
1,1960078,84(4Вел),На службе зла : роман,2017,Роберт Гэлбрейт ; перевод с английского Е. Пет...,Робин Эллакотт получает с курьером таинственны...,,роман,540,,mono,Русский,Текст,16.0,Художественная литература,41515.0,Иностранная литература. Современная классика,0dc0c51c9949ae797a04f1622dffb0f9,На службе зла,1
2,26991,84(7Сое),"Жареные зеленые помидоры в кафе ""Полустанок"" :...",2004,Фэнни Флэгг ; [пер. с англ. Д. Крупской],Роман американской писательницы Ф.Флэгг расска...,,роман,428,,mono,Русский,Текст,NaN,Художественная литература,116027.0,None,63e4b98dbc52b93a8d52540c1bed7c15,"Жареные зеленые помидоры в кафе ""Полустанок""",7
3,1781433,84(7Сое),Эмпайр Фоллз : роман,2020,Ричард Руссо ; перевод с английского Елены Пол...,"Майлз Роби двадцать лет готовит бургеры в ""Имп...",,роман,606,,mono,Русский,Текст,16.0,Художественная литература,141830.0,None,3333ebc7e54b7aab6b8e65f30cb2b4f4,Эмпайр Фоллз,11
4,1687060,84(7США),Эмпайр Фоллз : роман,2019,Р. Руссо ; пер. Е. Полецкая,,,роман,608,,mono,Русский,Текст,NaN,Художественная литература,141830.0,None,0f4b28cf531daa0af4662a85110fb249,Эмпайр Фоллз,1


In [25]:
books_info.to_csv(f'{PROJECT_FOLDER}/data/books.csv', index=False)

In [14]:
data = pd.read_csv(f'{PROJECT_FOLDER}/data/books.csv')
data.head()


,id,bbk,title,year,contribution,annotation,keyword,titleAdditionalInfo,volume,parallelTitle,publicationType,language_name,material_name,ageRestriction_name,rubric_name,author_id,serial_name,collapse_field,title_orig,available
0,1272541,84(2Рос=Рус)1,Бесы,2013,Федор Достоевский,"""Бесы""- одно из наиболее трагических, загадочн...",роман,NaN,604,NaN,mono,Русский,Текст,NaN,Художественная литература,45567.0,Русская классика,8b7355ec9ebf6ae5fdb84f917be6cc54,Бесы,2
1,1960078,84(4Вел),На службе зла : роман,2017,Роберт Гэлбрейт ; перевод с английского Е. Пет...,Робин Эллакотт получает с курьером таинственны...,NaN,роман,540,NaN,mono,Русский,Текст,16.0,Художественная литература,41515.0,Иностранная литература. Современная классика,0dc0c51c9949ae797a04f1622dffb0f9,На службе зла,1
2,26991,84(7Сое),"Жареные зеленые помидоры в кафе ""Полустанок"" :...",2004,Фэнни Флэгг ; [пер. с англ. Д. Крупской],Роман американской писательницы Ф.Флэгг расска...,NaN,роман,428,NaN,mono,Русский,Текст,NaN,Художественная литература,116027.0,NaN,63e4b98dbc52b93a8d52540c1bed7c15,"Жареные зеленые помидоры в кафе ""Полустанок""",7
3,1781433,84(7Сое),Эмпайр Фоллз : роман,2020,Ричард Руссо ; перевод с английского Елены Пол...,"Майлз Роби двадцать лет готовит бургеры в ""Имп...",NaN,роман,606,NaN,mono,Русский,Текст,16.0,Художественная литература,141830.0,NaN,3333ebc7e54b7aab6b8e65f30cb2b4f4,Эмпайр Фоллз,11
4,1687060,84(7США),Эмпайр Фоллз : роман,2019,Р. Руссо ; пер. Е. Полецкая,NaN,NaN,роман,608,NaN,mono,Русский,Текст,NaN,Художественная литература,141830.0,NaN,0f4b28cf531daa0af4662a85110fb249,Эмпайр Фоллз,1
